In [1]:
import json
import csv

training_info_file = "data\\train\\train_protein_information.json"
testing_info_file = "data\\test\\test_protein_information.json"

hypothetical_output = "data\\hypothetical_data.tsv"
training_output = "data\\training_data.tsv"

In [2]:
#test/hypothetical protein prep
with open(testing_info_file, 'r') as file:
    test_proteins_raw = json.load(file)

test_proteins_prep = {}
for id in test_proteins_raw.keys():
    test_proteins_prep[id] = {}

#training data protein prep
with open(training_info_file, 'r') as file:
    train_proteins_raw = json.load(file)

train_proteins_prep = {}
for id in train_proteins_raw.keys():
    train_proteins_prep[id] = {}

Names

In [3]:
for p_id, p_data in test_proteins_raw.items():
    test_proteins_prep[p_id]["name"] = p_data["name"]
#---
for p_id, p_data in train_proteins_raw.items():
    train_proteins_prep[p_id]["name"] = p_data["name"]

GO Terms

In [4]:
go_list = [
"GO:0000049",
"GO:0000287",
"GO:0003677",
"GO:0003723",
"GO:0005506",
"GO:0005524",
"GO:0005525",
"GO:0008270",
"GO:0016887",
"GO:0019843",
"GO:0030170",
"GO:0046872",
"GO:0050661",
"GO:0051287",
"GO:0051539",
]


#---------------------
# go_terms = {"mf": [], "cc": [], "bp": []}
# for p_id, p_data in proteins_raw.items():
#     for p_term in p_data["go_terms"]:
#         go_type = ""
#         match p_term[1][0]:
#             case "F": go_type = "mf"
#             case "P": go_type = "bp"
#             case "C": go_type = "cc"

#         if p_term[0] not in go_terms[go_type]:
#             go_terms[go_type].append(p_term[0])

# for aspect in go_terms.keys():
#     go_terms[aspect].sort()

In [5]:
for id in test_proteins_prep.keys():
    test_proteins_prep[id]["go_terms"] = {}


    for p_id, p_data in test_proteins_raw.items():
        p_encoded_go = [0] * len(go_list)
        for p_go in p_data["go_terms"]:
            if p_go[0] in go_list:
                index = go_list.index(p_go[0])
                p_encoded_go[index] = 1
        test_proteins_prep[p_id]["go_terms"] = p_encoded_go
#---
for id in train_proteins_prep.keys():
    train_proteins_prep[id]["go_terms"] = {}


    for p_id, p_data in train_proteins_raw.items():
        p_encoded_go = [0] * len(go_list)
        for p_go in p_data["go_terms"]:
            if p_go[0] in go_list:
                index = go_list.index(p_go[0])
                p_encoded_go[index] = 1
        train_proteins_prep[p_id]["go_terms"] = p_encoded_go



#---------------------
# for id in proteins_prep.keys():
#     proteins_prep[id]["go_terms"] = {}

# for aspect, terms in go_terms.items():
#     for p_id, p_data in proteins_raw.items():
#         p_encoded_go = [0] * len(go_terms[aspect])
#         for p_go in p_data["go_terms"]:
#             if p_go[0] in terms:
#                 index = terms.index(p_go[0])
#                 p_encoded_go[index] = 1
#         proteins_prep[p_id]["go_terms"][aspect] = p_encoded_go


In [6]:
# #test
# test_protein_terms = proteins_raw["A0A0X8BZN2"]["go_terms"]
# for term in test_protein_terms:
#     if "F:" in term[1]: print(f"F:{term[0]}: {go_terms['mf'].index(term[0])}")
#     elif "C:" in term[1]: print(f"C:{term[0]}: {go_terms['cc'].index(term[0])}")
#     elif "P:" in term[1]: print(f"P:{term[0]}: {go_terms['bp'].index(term[0])}")

# print()
# for aspect in go_terms.keys():
#     c = -1
#     for num in proteins_prep["A0A0X8BZN2"]["go_terms"][aspect]:
#         c+=1
#         if num == 1: print(c)
#     print()

Subcellular Location Feature

In [7]:
loc_code = {
    "0": ["Cytoplasm", "Cytoplasmic side", "Cytoplasm, nucleoid"],
    "1": ["Cell membrane", "Membrane", "Cell inner membrane", "Cell outer membrane", "Cytoplasmic side"],
    "2": ["Cell wall"],
    "3": ["Secreted"],
    "4": ["Periplasm", "Periplasmic side"],
    "5": ["Cell surface"],
    "6": ["Cell envelope"],
    "7": ["Chlorosome"],
    "8": ["Cellular thylakoid membrane"],
    "9": ["Cellular chromatopore membrane"],
    "10": ["Single-pass membrane protein"],
    "11": ["Multi-pass membrane protein"],
    "12": ["Peripheral membrane protein"]
}

test_locs = {}
for p_id, p_data in test_proteins_raw.items():
    p_loc_ids = []
    for p_loc in p_data["subcell_locations"]:
        for loc_id, loc_group in loc_code.items():
            if p_loc in loc_group:
                p_loc_ids.append(loc_id)
    test_locs[p_id] = p_loc_ids
#---
train_locs = {}
for p_id, p_data in train_proteins_raw.items():
    p_loc_ids = []
    for p_loc in p_data["subcell_locations"]:
        for loc_id, loc_group in loc_code.items():
            if p_loc in loc_group:
                p_loc_ids.append(loc_id)
    train_locs[p_id] = p_loc_ids


In [8]:
for p_id in test_proteins_raw.keys():
    p_encoded_locs = [0] * 13
    for p_loc in test_locs[p_id]:
        p_encoded_locs[int(p_loc)] = 1
    test_proteins_prep[p_id]["subcell_locations"] = p_encoded_locs
#---
for p_id in train_proteins_raw.keys():
    p_encoded_locs = [0] * 13
    for p_loc in train_locs[p_id]:
        p_encoded_locs[int(p_loc)] = 1
    train_proteins_prep[p_id]["subcell_locations"] = p_encoded_locs

In [9]:
#test
for loc in test_proteins_raw["A0A0X8C1K8"]["subcell_locations"]:
    print(loc)

print()

for loc in test_proteins_prep["A0A0X8C1K8"]["subcell_locations"]:
    print(loc)

#---
for loc in train_proteins_raw["A5FB63"]["subcell_locations"]:
    print(loc)

print()

for loc in train_proteins_prep["A5FB63"]["subcell_locations"]:
    print(loc)

Cell membrane
Cytoplasm

1
1
0
0
0
0
0
0
0
0
0
0
0
Secreted

0
0
0
1
0
0
0
0
0
0
0
0
0


Binding Preference

In [10]:
bp_code = {
    "0": "DNA/RNA-binding",
    "1": "Nucleotide-binding",
    "2": "Metal-binding",
}


test_bps = {}
for p_id, p_data in test_proteins_raw.items():
    p_bp_ids = []
    for p_bp in p_data["binding_preference"]:
        for bp_id, bp_name in bp_code.items():
            if p_bp == bp_name:
                p_bp_ids.append(bp_id)
    test_bps[p_id] = p_bp_ids
#---
train_bps = {}
for p_id, p_data in train_proteins_raw.items():
    p_bp_ids = []
    for p_bp in p_data["binding_preference"]:
        for bp_id, bp_name in bp_code.items():
            if p_bp == bp_name:
                p_bp_ids.append(bp_id)
    train_bps[p_id] = p_bp_ids

In [11]:
for p_id in test_proteins_raw.keys():
    p_encoded_bps = [0] * 3
    for p_bp in test_bps[p_id]:
        p_encoded_bps[int(p_bp)] = 1
    test_proteins_prep[p_id]["binding_preference"] = p_encoded_bps
#---
for p_id in train_proteins_raw.keys():
    p_encoded_bps = [0] * 3
    for p_bp in train_bps[p_id]:
        p_encoded_bps[int(p_bp)] = 1
    train_proteins_prep[p_id]["binding_preference"] = p_encoded_bps

Motifs (DO NOT USE) - All motifs collected, too many features. Unsure of how consensus patterns work.

In [12]:
# motifs = []
# for p_id, p_data in proteins_raw.items():
#     for motif in p_data["motifs"]:
#         if motif not in motifs:
#             motifs.append(motif)

# motifs.sort()
# num_motifs = len(motifs)

In [13]:
# for p_id, p_data in proteins_raw.items():
#     p_encoded_motifs = [0] * num_motifs
#     for p_motif in p_data["motifs"]:
#         index = motifs.index(p_motif)
#         p_encoded_motifs[index] = 1
#     proteins_prep[p_id]["motifs"] = p_encoded_motifs

In [14]:
# #test
# test_protein_motifs = proteins_raw["A0A0X8BZN2"]["motifs"]
# for motif in test_protein_motifs:
#     print(f"{motif}: {motifs.index(motif)}")
# print()
# c = -1
# for num in proteins_prep["A0A0X8BZN2"]["motifs"]:
#     c+=1
#     if num == 1: print(c)

Simple Features

In [15]:
features = ["length", "m_weight", "instab_index", "isoele_point", "gravy", "sec_sruct_frac", "ext_coeff", "transmembrane"]
for p_id, p_data in test_proteins_raw.items():
    for feature in features:
        test_proteins_prep[p_id][feature] = p_data[feature]
#---
for p_id, p_data in train_proteins_raw.items():
    for feature in features:
        train_proteins_prep[p_id][feature] = p_data[feature]


In [16]:
#test
for feature in features:
    print(test_proteins_raw["A0A0X8BZN2"][feature])

print()

for feature in features:
    print(test_proteins_prep["A0A0X8BZN2"][feature])

#---
for feature in features:
    print(train_proteins_raw["A5FB63"][feature])

print()

for feature in features:
    print(train_proteins_prep["A5FB63"][feature])


    

423
48995.48860000003
40.19888888888888
6.514680671691894
-0.3872340425531913
[0.33333333333333337, 0.21276595744680848, 0.26004728132387706]
[83310, 83560]
0

423
48995.48860000003
40.19888888888888
6.514680671691894
-0.3872340425531913
[0.33333333333333337, 0.21276595744680848, 0.26004728132387706]
[83310, 83560]
0
1578
168919.30859999952
20.26636882129288
6.3476877212524405
-0.08891001267427177
[0.31368821292775667, 0.30354879594423323, 0.2002534854245881]
[231150, 231775]
0

1578
168919.30859999952
20.26636882129288
6.3476877212524405
-0.08891001267427177
[0.31368821292775667, 0.30354879594423323, 0.2002534854245881]
[231150, 231775]
0


Amino Count

In [17]:
for p_id, p_data in test_proteins_raw.items():
    acids = []
    for acid in p_data["amino_count"].values():
        acids.append(acid)
    test_proteins_prep[p_id]["amino_count"] = acids
#---
for p_id, p_data in train_proteins_raw.items():
    acids = []
    for acid in p_data["amino_count"].values():
        acids.append(acid)
    train_proteins_prep[p_id]["amino_count"] = acids

In [18]:
#test
for acid in test_proteins_raw["A0A0X8BZN2"]["amino_count"].values():
    print(acid)

print()

for acid in test_proteins_prep["A0A0X8BZN2"]["amino_count"]:
    print(acid)

#---

for acid in train_proteins_raw["A5FB63"]["amino_count"].values():
    print(acid)

print()

for acid in train_proteins_prep["A5FB63"]["amino_count"]:
    print(acid)


29
5
17
33
25
27
13
29
30
37
11
26
20
22
17
17
15
21
10
19

29
5
17
33
25
27
13
29
30
37
11
26
20
22
17
17
15
21
10
19
141
10
72
30
63
126
15
67
73
122
23
140
76
50
24
137
166
139
19
85

141
10
72
30
63
126
15
67
73
122
23
140
76
50
24
137
166
139
19
85


iFeature

In [20]:
for p_id, p_data in test_proteins_raw.items():
    for feature in ["dpc", "paac", "ctdc", "ctdt", "ctdd", "ctriad", "gaac", "moran"]:
        values = []
        for value in p_data[feature].values():
            values.append(value)
        test_proteins_prep[p_id][feature] = values
#---
for p_id, p_data in train_proteins_raw.items():
    for feature in ["dpc", "paac", "ctdc", "ctdt", "ctdd", "ctriad", "gaac", "moran"]:
        values = []
        for value in p_data[feature].values():
            values.append(value)
        train_proteins_prep[p_id][feature] = values

Extract hypotheticals 

In [21]:
hypothetical_proteins = {}

for id, data in test_proteins_prep.items():
    if "hypothetical protein" in data["name"]:
        hypothetical_proteins[id] = data

OUTPUT

In [22]:
def output_proteins(proteins, outfile):
    rows = []

    header = ["protein_id"]
    r1 = proteins[next(iter(proteins))]
    for feature, values in r1.items():
        if feature == "go_terms": continue
        if type(values) == list:
            for i in range(0,len(values)):
                header.append(f"{feature}{i}")
        else:
            header.append(feature)
    for term in go_list:
        header.append(term)
    rows.append(header)


    for p_id, p_data in proteins.items():
        row = [p_id]
        for feature, value in p_data.items():
            if feature == "go_terms": continue
            if type(value) == list:
                for i in range(0,len(value)):
                    row.append(value[i])
            else:
                row.append(value)
        for value in p_data["go_terms"]:
            row.append(value)
        rows.append(row)

    with open(outfile, "w", newline="") as file:
        writer = csv.writer(file, delimiter="\t")
        writer.writerows(rows)

output_proteins(hypothetical_proteins, hypothetical_output)
output_proteins(train_proteins_prep, training_output)